In [ ]:
import cv2

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from PIL import Image, ImageDraw, ImageFilter, ImageOps, ImageEnhance


import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
CUSTOM_MODEL_NAME = '' 
TF_RECORD_SCRIPT_NAME = ''
LABEL_MAP_NAME = ''


paths = {
   
    'CHECKPOINT_PATH': os.path.join('PATH',CUSTOM_MODEL_NAME),  
    'ANNOTATION_PATH': os.path.join('PATH','annotations'),
    'IMAGE_PATH': os.path.join('PATH'),
    'SCRIPTS_PATH': os.path.join('PATH','tensorflow','scripts'),
 }


files = {
    'PIPELINE_CONFIG':os.path.join('PATH', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'CHECKPOINT')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'IMAGE_FOR_DETECTION')

In [ ]:
img = Image.open(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)


In [ ]:
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()


In [ ]:
viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,            
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=50,
            min_score_thresh=0.25,
            line_thickness=1,
            skip_labels=True,
            agnostic_mode=False)


picture= cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
cv2.imwrite('detection.jpg', picture)

plt.figure(figsize=(40,40))
plt.imshow(picture)
plt.show()

In [ ]:
detection_threshold = 0.25
image = cv2.imread(IMAGE_PATH)

 
scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
boxes = detections['detection_boxes'][:len(scores)]
classes = detections['detection_classes'][:len(scores)]

width = image.shape[1]
height = image.shape[0]

In [ ]:
liste=[]
m=0


while m<len(classes):
    if classes[m]==0 or classes[m]==1:
        liste.append(m)
    
    
    m+=1
 
print(liste)

In [ ]:
scores_filtered=[]

for i in liste:
    scores_filtered.append(scores[i])

    
print(scores_filtered)

In [ ]:
boxes_filtered=[]

for i in liste:
    boxes_filtered.append(boxes[i])
    
print(boxes_filtered)    

In [ ]:
i=0
outputs=[]
text=''

while i<len(scores_filtered):
    roi = boxes_filtered[i]*[height, width, height, width]        
    region = image[int(roi[0])-5:int(roi[2])+5,int(roi[1])-5:int(roi[3])+5]
    bild = region.shape
    
    if region.shape[0]>region.shape[1]+5:
        region = np.rot90(region,-1)
        
    img = cv2.cvtColor(region, cv2.COLOR_BGR2RGB)        
    cv2.imwrite('detection_ocr%s.jpg'% i, img)
    i+=1